In [6]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [23]:
%sendtofile main.py

import machine, time
a1 = machine.ADC(machine.Pin(32))
a1.atten(machine.ADC.ATTN_11DB)
a1.width(machine.ADC.WIDTH_12BIT)

b = machine.PWM(machine.Pin(23, machine.Pin.OUT))
d = machine.PWM(machine.Pin(22, machine.Pin.OUT), duty=102)

x0 = 0
while 1:
    x = a1.read()
    if abs(x-x0) > 2:
        f = x/4095
        b.duty(int(f*512+511))
        d.duty(int(f*150+130))
        x0 = x
        print(x)
    time.sleep(0.2)


Sent 18 lines (422 bytes) to main.py.


In [44]:
# This requires you to checkout https://github.com/micropython/micropython-lib and avoids the upip
%sendtofile --mkdir --source /home/julian/extrepositories/micropython-lib/uasyncio/uasyncio/__init__.py lib/uasyncio/
%sendtofile --mkdir --source /home/julian/extrepositories/micropython-lib/uasyncio.core/uasyncio/core.py lib/uasyncio/
%sendtofile --mkdir --source /home/julian/extrepositories/micropython-lib/uasyncio.queues/uasyncio/queues.py lib/uasyncio/
%sendtofile --mkdir --source /home/julian/extrepositories/micropython-lib/uasyncio.synchro/uasyncio/synchro.py lib/uasyncio/

# This requires you to checkout https://github.com/peterhinch/micropython-async
%sendtofile --source /home/julian/extrepositories/micropython-async/asyn.py
%sendtofile --source /home/julian/extrepositories/micropython-async/aswitch.py

# This requires you to checkout https://github.com/peterhinch/micropython-mqtt/tree/master/mqtt_as
#%sendtofile --source /home/julian/extrepositories/micropython-mqtt/mqtt_as/mqtt_as.py
# Forked version https://github.com/goatchurchprime/micropython-mqtt
%sendtofile --source /home/julian/repositories/micropython-mqtt/mqtt_as/mqtt_as.py

%sendtofile --source ../Sensor_Kennel/sevensegmentdisplay.py
#%sendtofile --source ../basicsockets/minimalmdns.py


Sent 79 lines (2294 bytes) to sevensegmentdisplay.py.


In [9]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [10]:
%sendtofile main.py

from sevensegmentdisplay import encodeledstring
from machine import I2C, Pin
import time

i2c = I2C(scl=Pin(15, Pin.OUT, Pin.PULL_UP), sda=Pin(4, Pin.OUT, Pin.PULL_UP), freq=100000)

tempAmbient, tempIR = 0, 0
tempBad = True
tempTime = 0

def checkCRC(k, mask):  # mask (matching nibble) different in case
    inCrc = 0
    for inData in [k[0], k[1]]:
        data = inCrc ^ inData;
        data <<= 8 
        for i in range(8):
            if (( data & 0x8000 ) != 0 ):
                data = data ^ (0x1070 << 3)
            data = data << 1
        inCrc = data >> 8
    return (inCrc & mask) == (k[2]&mask)


async def readtemps():
    global ntempcount
    while True:
        global tempAmbient, tempIR, tempBad, tempTime
        try:
            ka = i2c.readfrom_mem(0x5a, 0x06, 3)
            ki = i2c.readfrom_mem(0x5a, 0x07, 3)
            bcrcgood = (checkCRC(ka, 0x0F) and checkCRC(ki, 0xF0))
            print(bcrcgood, ka, ki)
            if not bcrcgood:
                raise OSError()
            tempAmbient = (ka[0]+ka[1]*256)*0.02 - 273.15
            tempIR = (ki[0]+ki[1]*256)*0.02 - 273.15
            tempBad = False
        except OSError:
            tempBad = True
            
        if not tempBad:
            encodeledstring("%5.2f %.1f"% (tempIR, tempAmbient))
        else:
            encodeledstring("bad %04d"%(tempTime%10000))
        tempTime = time.ticks_ms()
        await asyncio.sleep_ms(200)

        
from mqtt_as import MQTTClient
import uasyncio as asyncio
import asyn
from sevensegmentdisplay import encodeledstring, writeledstringautoscroll

encodeledstring("start up")

import uasyncio as asyncio
import asyn
from mqtt_as import config

# Include any cross-project settings.
config['server'] = "mqtt.local"
config['ssid'] = 'DoESLiverpool'
config['wifi_pw'] = 'decafbad00'
config['mqttchannel'] = "whitesp"

mqttchannelA = config['mqttchannel']+'/tempambient'
mqttchannelI = config['mqttchannel']+'/tempir'
mqttchannelIP = config['mqttchannel']+'/ip'
mqttchannelBad = config['mqttchannel']+'/tempoutofrange'

async def main(client):
    DtempTime = 0
    await client.connect()
    for i in range(10):
        encodeledstring("connect%d"%i)
        await asyncio.sleep_ms(100)
        
    n = 0
    while True:
        if DtempTime != tempTime:
            if not tempBad:  # no inconsistency problem on this as async
                if -10<tempAmbient<40:
                    await client.publish(mqttchannelA, '%.2f'%tempAmbient, qos=1)
                else:
                    await client.publish(mqttchannelBad, 'A %.2f'%tempAmbient, qos=1)

            if not tempBad:  # no inconsistency problem on this as async
                if -10<tempIR<40:
                    await client.publish(mqttchannelI, '%.2f'%tempIR, qos=1)
                else:
                    await client.publish(mqttchannelBad, 'I %.2f'%tempIR, qos=1)
            DtempTime = tempTime
        n += 1
        if (n%10) == 0:
            await client.publish(mqttchannelIP, client._sta_if.ifconfig()[0], retain=True, qos=1)
        await asyncio.sleep_ms(100)


MQTTClient.DEBUG = True  # Optional: print diagnostic messages
client = MQTTClient(config)
aloop = asyncio.get_event_loop()
aloop.create_task(readtemps())
encodeledstring("ready.")
msg = "fail."
try:
    aloop.run_until_complete(main(client))
except Exception as e:
    print(e)
    msg = str(e).lower()
client.close()  # Prevent LmacRxBlk:1 errors

encodeledstring(msg)
while 1:
    writeledstringautoscroll()


Sent 114 lines (3494 bytes) to main.py.


In [24]:
print(client.port)

1883
